In [1]:
import pandas as pd

### CSV 불러오기

In [3]:
df = pd.read_csv("/Users/suchan/study/파이널 프로젝트/0426_KOR_sentiment.csv")

### 전처리

In [9]:
import re
from konlpy.tag import Mecab
from tqdm import tqdm

In [10]:
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern, '', text)
    return text

def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    stop_words = ['드라마', '정도', '사람', '생각', '여자', '남자', '에피소드', '시리즈', '자신', '최고', '완벽', '시즌', '느낌', '기대',
                  '넷플릭스', '장르']
    nouns = [word for word, tag in tagged if tag in ['NNG', 'NNP'] and len(word) >1 and word not in stop_words]
    return nouns

def tokenize(df):
    tokenizer = Mecab()
    processed_data = []
    for sent in tqdm(df['reviews']):
        sentence = clean_text(str(sent).replace("\n", "").strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data

In [11]:
preprocessed_data = tokenize(df)

100%|██████████| 62928/62928 [00:08<00:00, 7387.34it/s]


In [12]:
df['preprocessed_reviews'] = preprocessed_data

In [13]:
df['preprocessed_reviews']

0                             [캐스팅, 라인업, 노희경, 극본, 몰입, 차승원]
1                                    [침묵, 순간, 노희경, 극본, 경지]
2                                   [세상, 노희경, 작가, 배우, 스토리]
3                                         [배우, 한수, 위험, 고통]
4                                            [특별, 저마다, 사정]
                               ...                        
62923    [종교, 재료, 집단, 자살, 사건, 쇼킹, 무당, 소재, 순간, 답지, 문제, 기...
62924    [김단, 천재인, 캐릭터, 이용, 단편, 매력, 연계, 주요, 갈등, 맥락, 상대,...
62925                      [흥미, 중반, 완성도, 후반, 마지막, 아빠, 피해자]
62926                                 [대본, 연출, 연기, 모두, 부족]
62927                               [국판, 데스노트, 천재, 심리, 싸움]
Name: preprocessed_reviews, Length: 62928, dtype: object

### 드라마 별 DataFrame 준비

In [7]:
drama_list = df.title.unique().tolist()

In [24]:
drama1 = [['노희경', '차승원', '극본', '작가', '아쉬움'], ['인생', '사람', '행복', '마음'], ['연출', '김우빈', '수지', '연기']]
sum(drama1, [])

['노희경',
 '차승원',
 '극본',
 '작가',
 '아쉬움',
 '인생',
 '사람',
 '행복',
 '마음',
 '연출',
 '김우빈',
 '수지',
 '연기']

In [28]:
each_drama_reviews_dict = {'title' : [], 'preprocessed_reviews' : []}
for drama in tqdm(drama_list):
    drama_review_list = df[df['title'] == drama].preprocessed_reviews.tolist()
    reviews_join = sum(drama_review_list,[])
    each_drama_reviews_dict['title'].append(drama)
    each_drama_reviews_dict['preprocessed_reviews'].append(reviews_join)

100%|██████████| 92/92 [00:02<00:00, 40.78it/s]


In [30]:
df_each_drama_reviews = pd.DataFrame(each_drama_reviews_dict)

In [32]:
df_each_drama_reviews.to_csv("/Users/suchan/study/파이널 프로젝트/each_drama_reviews.csv", index=False)

In [33]:
df_each_drama_reviews

,title,preprocessed_reviews
0,우리들의 블루스,"[캐스팅, 라인업, 노희경, 극본, 몰입, 차승원, 침묵, 순간, 노희경, 극본, ..."
1,사내 맞선,"[안효섭, 지현우, 전편, 감상, 소재, 파리, 연인, 만화, 연출, 여주인공, 조..."
2,스물다섯 스물하나,"[남주, 연기, 이상, 몰입, 작품, 안목, 상대, 배우, 주연, 배우, 연기, 버..."
3,시맨틱 에러,"[때깔, 캐스팅, 찰떡, 캠퍼, 로코, 분량, 캐스팅, 찰떡, 추상, 회차, 최유,..."
4,악의 마음을 읽는 자들,"[전형, 재미, 클리셰, 떡칠, 희석, 불안, 마음, 전형, 범죄, 수사, 편견, ..."
...,...,...
87,자백,"[반전, 뒷부분, 결말, 스마트, 유성주, 배우, 스카이, 캐슬, 새끼, 연기, 류..."
88,월간 집,"[전개, 감정, 약간, 지루함, 캐릭터, 설정, 별점, 일념, 동생, 코미디, 현실..."
89,시를 잊은 그대에게,"[선생, 보영, 시청자, 힐링, 작품, 코믹, 요소, 전체, 보영, 이준혁, 인생,..."
90,본 대로 말하라,"[답답이, 고구마, 조직, 조직, 대장, 장혁, 이하나, 보이스, 놀이, 악역, 설..."
